In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

# LAB CASE REGRESSION

In [2]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)

In [3]:
data.head(35)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3

In [4]:
data_donations = data[data['TARGET_D']!=0].reset_index(drop=True)

In [5]:
y = data_donations['TARGET_D']
X = data_donations.drop(['TARGET_B', 'TARGET_D'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

In [6]:
# OneHotEncoding of the categoricals
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)
X.shape

(4843, 354)

In [7]:
# Train/Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

In [8]:
# Data Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [9]:
# Format the data as dataframes
X_train_scaled = pd.DataFrame(X_train_scaled).reset_index(drop=True)
X_train_scaled.columns = X_train.columns
X_test_scaled = pd.DataFrame(X_test_scaled).reset_index(drop=True)
X_test_scaled.columns = X_test.columns

# I separate original numericals and categorical columns to check the variance of the numerical data
X_train_scaled_cat = X_train_scaled[list(encoded_categorical.columns)].reset_index(drop=True)
X_train_scaled_num = X_train_scaled.drop(list(encoded_categorical.columns), axis=1)

In [10]:
X_test_scaled

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.000026,0.723404,0.166667,1.000000,0.000000,0.010417,0.430380,0.242424,0.505051,0.125000,0.20,0.033333,1.000000,0.135252,0.168474,0.124334,0.000000,0.070707,0.939394,0.494949,0.671053,0.868687,0.131313,0.011111,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.506329,0.620253,0.6625,0.539474,0.657895,0.697368,0.295775,0.383838,0.383838,0.242424,0.109890,0.171717,0.171717,0.352941,0.358974,0.384615,0.166667,0.151515,0.484848,0.500000,0.46,0.161616,0.397959,0.146341,0.381443,0.234043,0.505882,0.343434,0.191919,0.070707,0.020202,0.666667,0.080808,0.160714,0.184783,0.252308,0.344286,0.767677,0.757576,0.000000,0.050505,0.040404,0.020202,0.020202,0.025974,0.000000,0.158643,0.174096,0.153846,0.153846,0.828283,0.181818,0.909091,0.102041,0.141414,0.292929,0.757576,0.646465,0.222222,0.868687,0.141414,0.120690,0.1,0.089286,0.134146,0.240000,0.035714,0.197183,0.444444,0.288660,0.069767,0.115942,0.060606,0.060606,0.191919,0.474747,0.636364,0.808081,0.010101,0.011111,0.545455,0.666667,0.647727,0.213115,0.285714,0.032967,0.020202,0.191919,0.101010,0.028169,0.020202,0.086957,0.131313,0.171717,0.313131,0.565657,0.000000,0.544186,0.635641,0.202000,0.231333,0.248924,0.279770,0.082791,0.216495,0.363636,0.361702,0.372881,0.141414,0.056604,0.034483,0.047619,0.014286,0.141414,0.161616,0.333333,0.252525,0.171717,0.095238,0.032258,0.043478,0.014286,0.414141,0.090909,0.432990,0.125000,0.424242,0.623656,0.147368,0.828283,0.121212,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.102564,0.013333,0.108696,0.188889,0.377358,0.04,0.626263,0.555556,0.606061,0.515152,0.595960,0.494949,0.757576,0.595960,0.838384,1.000000,0.040404,0.153846,0.20,0.020202,0.164179,0.241379,0.043478,0.04,0.230769,0.057143,0.270833,0.243902,0.10,0.16,0.042857,0.000000,0.187500,0.349206,0.050000,0.095238,0.08,0.188235,0.075472,0.063830,0.166667,0.032787,0.070707,0.125000,0.092593,0.114286,0.166667,0.18,0.033333,0.140845,0.676768,0.127660,0.0,0.705882,0.080808,0.232143,0.441176,0.466667,0.290323,0.25000

### Removing columns with low variance

In [11]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02   # put at same level as for classification lab
sel = VarianceThreshold(threshold=(var_threshold))

sel = sel.fit(X_train_scaled_num)
temp = sel.transform(X_train_scaled_num)
temp = pd.DataFrame(temp)
print(temp.shape)

(3874, 104)


In [12]:
var_list = list(sel.get_support())
cols_to_drop = [col[0] for col in zip(X_train_scaled_num.columns, var_list) if col[1] == False]

In [13]:
X_train_scaled_and_cleaned_num = pd.DataFrame(X_train_scaled_num, columns=X_train_scaled_num.columns).drop(cols_to_drop, axis=1).reset_index(drop=True)

# put back together numerical and categorical for the train set
X_train_scaled_and_cleaned = pd.concat([X_train_scaled_and_cleaned_num, X_train_scaled_cat], axis=1)

# Remove drop_columns from the test set
X_test_scaled_and_cleaned = X_test_scaled.drop(cols_to_drop, axis=1).reset_index(drop=True)

print(X_train_scaled_and_cleaned.shape)
print(X_test_scaled_and_cleaned.shape)

(3874, 128)
(969, 128)


### Regression model - Decision Tree Regressor

In [14]:
regr = DecisionTreeRegressor()
model = regr.fit(X_train_scaled_and_cleaned, y_train)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [15]:
model.score(X_train_scaled_and_cleaned, y_train)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


1.0

In [16]:
model.score(X_test_scaled_and_cleaned, y_test)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


-0.37177227149723446

This does not work and I do not understand why... I 'll try another regression model.

### Regression model - Linear Regression with application of Recursive Feature Elimination

In [17]:
from sklearn.feature_selection import RFE # Recursive Feature Elimination
from sklearn import linear_model
lm = linear_model.LinearRegression()
rfe = RFE(lm, n_features_to_select=20, verbose=False)
rfe.fit(X_train_scaled_and_cleaned, y_train)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RFE(estimator=LinearRegression(), n_features_to_select=20, verbose=False)

In [20]:
# After we run the algorithm, it labels the top features as 1 and the rest are marked in an increasing order of importance.
df = pd.DataFrame(data = rfe.ranking_, columns=['Rank'])
df['Column_name'] = pd.DataFrame(X_train_scaled_and_cleaned).columns
df.head(60)

,Rank,Column_name
0,73,AGE
1,52,INCOME
2,87,WEALTH1
3,33,MALEVET
4,68,VIETVETS
5,35,WWIIVETS
6,64,WEALTH2
7,1,POP90C1
8,1,POP90C2
9,1,POP90C3


In [23]:
main_features = df[df['Rank']==1]['Column_name']
main_features = list(main_features)
main_features

['POP90C1',
 'POP90C2',
 'POP90C3',
 'DW1',
 'DW6',
 'HU1',
 'HU2',
 'HUPA2',
 'HUPA3',
 'HUPA6',
 'IC6',
 'IC7',
 'HHAS1',
 'MC1',
 'MC2',
 'HC17',
 'HC18',
 'RFA_2F',
 15,
 16]

In [25]:
X_train_main = X_train_scaled_and_cleaned[main_features]
X_test_main = X_test_scaled_and_cleaned[main_features]

In [27]:
model = lm.fit(X_train_main, y_train)
lm_pred = lm.predict(X_test_main)
lm.score(X_test_main, y_test)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.36118364159372185

In [28]:
lm.score(X_train_main, y_train)

C:\Users\ebour\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.37692243831404704

The score is pretty bad, but I really do not see how to go further... I am lost in all the selection processes and how to apply them on a mix of numerical and encoded categorical data...

### Estimation of the donations

In [32]:
predicted_donors = pd.read_csv('data_predicted.csv').drop(['Unnamed: 0'], axis=1)
predicted_donors.shape

(164037, 119)

In [33]:
X_train_main.shape

(3874, 20)

In [34]:
predicted_donors_regr = predicted_donors[main_features]

KeyError: "['IC7', 'HHAS1', 15, 16] not in index"

Here we are, both sets are built in different notebooks, usming different processes and I do not see how to apply my regression model, as I do not have all the necessary features...